In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers accelerate>=0.14.0 diffusers>=0.7.2 ftfy

In [3]:
import torch
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler, AutoencoderKL, UNet2DConditionModel, DDPMScheduler
from transformers import CLIPTextModel, CLIPTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
repo = "Bingsu/my-korean-stable-diffusion-v1-5"
euler_ancestral_scheduler = EulerAncestralDiscreteScheduler.from_config(repo, subfolder="scheduler")
noise_scheduler = DDPMScheduler.from_pretrained(repo, subfolder="scheduler")
vae = AutoencoderKL.from_pretrained(repo, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(repo, subfolder="unet")

text_encoder = CLIPTextModel.from_pretrained(repo, subfolder="text_encoder")
tokenizer = CLIPTokenizer.from_pretrained(repo, subfolder="tokenizer")

/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:240: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_euler_ancestral_discrete.EulerAncestralDiscreteScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)


scheduler/scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer/tokenizer_config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/870k [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

tokenizer/tokenizer.json:   0%|          | 0.00/2.91M [00:00<?, ?B/s]

In [5]:
device = "cuda"
text_encoder.to(device)
vae.to(device)
unet.to(device)

vae.requires_grad_(False)
text_encoder.requires_grad_(False)

CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e

In [ ]:
# 데이터로더 짜주세요 토크나이저 여기서 써야될듯

In [6]:
import pandas as pd
import tqdm
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
import torch as th
from torchvision import transforms

path = '/content/drive/MyDrive/인지프 7팀/데이터 수집/small_data.xlsx'

df = pd.read_excel(path)
df.columns = ["index", "title", "author", "gender", "period", "theme", "contents", "is_textbook", "publisher", "image"]

In [7]:
def preprocess_image(img) -> th.Tensor:
    transform = transforms.Compose([
        transforms.Resize(64),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    img = transform(img)
    return img

In [10]:
text = []
imgs = []

# iterrows()를 사용하여 각 행을 반복
for index, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    # 'contents' 컬럼의 값을 text 리스트에 추가
    text.append(row['title'] + ", " +row['contents'])

    # 'image' 컬럼의 값을 img_path로 저장하고 출력
    img_path = f"/content/drive/MyDrive/인지프 7팀/데이터 수집/image{row['image'][1:]}"
    img = Image.open(img_path).convert('RGB')
    imgs.append(preprocess_image(img))

imgs = th.stack(imgs, dim=0)

100%|██████████| 64/64 [00:01<00:00, 37.37it/s]


In [29]:
def split_dataset(encoder, tokenizer, text, imgs):
    # not use category now
    text_embs = []
    for e in tqdm.tqdm(text):
        encoded_text = tokenizer(e, padding="max_length", truncation=True, max_length=tokenizer.model_max_length, return_tensors="pt")
        text_emb = encoder(encoded_text.input_ids.to(device))[0].squeeze(0)
        text_embs.append(text_emb)
    text_embs = th.stack(text_embs, dim=0)

    k = int(len(text_embs)*0.8)
    train_dataset = TensorDataset(text_embs[:k], imgs[:k])
    test_dataset = TensorDataset(text_embs[k:], imgs[k:])

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

    return train_loader, test_loader

In [30]:
train_loader, test_loader = split_dataset(text_encoder, tokenizer, text, imgs)

  9%|▉         | 6/64 [00:00<00:01, 57.91it/s]

torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])


 38%|███▊      | 24/64 [00:00<00:00, 56.59it/s]

torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])


 56%|█████▋    | 36/64 [00:00<00:00, 56.21it/s]

...

torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])


 95%|█████████▌| 61/64 [00:01<00:00, 57.68it/s]

torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])


100%|██████████| 64/64 [00:01<00:00, 56.34it/s]

torch.Size([77, 768])
torch.Size([77, 768])
torch.Size([77, 768])


In [45]:
text_embs = []
for e in tqdm.tqdm(text):
    encoded_text = tokenizer(e, padding="max_length", truncation=True, max_length=tokenizer.model_max_length, return_tensors="pt")
    text_emb = text_encoder(encoded_text.input_ids.to(device))[0]
    text_embs.append(text_emb)
text_embs = th.stack(text_embs, dim=0)

100%|██████████| 64/64 [00:02<00:00, 23.16it/s]


In [46]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, texts, images):
        self.texts = texts
        self.images = images

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        image = self.images[idx]
        return {'input_ids': text, 'images': image}

# 이미 생성된 text와 imgs 리스트를 사용하여 데이터셋을 생성합니다.
dataset = CustomDataset(text_embs, imgs)

# 데이터셋을 훈련 및 테스트 세트로 분할합니다.
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# DataLoader를 생성합니다.
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# 이제 DataLoader를 사용하여 배치 데이터를 반복하면 딕셔너리 형태로 데이터를 얻을 수 있습니다.

In [13]:
prompt_vector = torch.randn(4, 768)
prompt_vector.requires_grad = True
category_prompt = prompt_vector[0]

In [14]:
first_epoch = 0
num_train_epochs = 10

optimizer_cls = torch.optim.AdamW

optimizer = optimizer_cls(
        [
            {'params': unet.parameters()},
            {'params': prompt_vector}
        ],
        lr=1e-4,
        betas=(0.9, 0.999),
        weight_decay=1e-2,
        eps=1e-08,
    )

In [64]:
!CUDA_LAUNCH_BLOCKING=1

In [31]:
import torch.nn.functional as F
# vae.encoder.half()

for epoch in range(first_epoch, num_train_epochs):
    unet.train()
    train_loss = 0.0
    for step, (X, y) in enumerate(train_loader):
        print(X.shape)
        with torch.no_grad():
            latents = vae.encoder(y.to(torch.float32).to(device))
            latents = latents * vae.config.scaling_factor
            transition_conv = torch.nn.Conv2d(in_channels=8, out_channels=4, kernel_size=1, stride=1, padding=0).to(device)
            latents = transition_conv(latents)


        noise = torch.randn_like(latents)
        bsz = latents.shape[0]

        timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device)
        timesteps = timesteps.long()
        print(latents.shape, noise.shape, timesteps.shape, encoder_hidden_states.shape)
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
        print(noisy_latents.shape)

        encoder_hidden_states = X
        # encoder_hidden_states = text_encoder(
        #             batch["input_ids"]
        #         )[0]

        if noise_scheduler.config.prediction_type == "epsilon":
            target = noise
        elif noise_scheduler.config.prediction_type == "v_prediction":
            target = noise_scheduler.get_velocity(latents, noise, timesteps)

        model_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

        loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")
        loss.backward()
        optimizer.step()



torch.Size([16, 77, 768])
torch.Size([16, 4, 8, 8]) torch.Size([16, 4, 8, 8]) torch.Size([16]) torch.Size([16, 1, 77, 768])
torch.Size([16, 4, 8, 8])


OutOfMemoryError: ignored